In [ ]:
#objective 1
#extract text from FIR images 

# Generic Libraries
from PIL import Image
import os
import pandas as pd
import numpy as np
import re,string,unicodedata

#Tesseract Library
import pytesseract

#Warnings
import warnings
warnings.filterwarnings("ignore")

#Garbage Collection
import gc

import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import pytesseract

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
custom_config = ' '

In [ ]:
# Path to the folder containing the images
folder_path = "FIR_images_v1"

# Get the list of files in the folder
image_files = os.listdir(folder_path)

# Custom Tesseract configuration if needed
custom_config = r'--oem 3 --psm 6'

# Iterate over each image file
for image_file in image_files:
    # Construct the full path to the image
    image_path = os.path.join(folder_path, image_file)
    
    # Load the image using OpenCV
    img = cv2.imread(image_path)
    
    # Perform OCR using pytesseract
    text = pytesseract.image_to_string(img, config=custom_config)

    # the output of OCR can be saved in a file in necessary
    file = open('output_final.txt','a') # file opened in append mode
    file.write(text)
    file.close()

In [ ]:
#objective 2
 
# just to show how spacy works 

import spacy

# Load English tokenizer, tagger, parser and NER
nlp = spacy.load("en_core_web_sm")

# Process whole documents
text = ("When Sebastian Thrun started working on self-driving cars at "
        "Google in 2007, few people outside of the company took him "
        "seriously. “I can tell you very senior CEOs of major American "
        "car companies would shake my hand and turn away because I wasn’t "
        "worth talking to,” said Thrun, in an interview with Recode earlier "
        "this week.")
doc = nlp(text)

# Analyze syntax
print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])
print("Verbs:", [token.lemma_ for token in doc if token.pos_ == "VERB"])

# Find named entities, phrases and concepts
for entity in doc.ents:
    print(entity.text, entity.label_)


In [ ]:
# actual code for legal keyword recognization from extracted text 

import spacy
import warnings

# Suppress specific warning
warnings.filterwarnings("ignore", message="Model 'en_blackstone_proto' .*")

try:
    # Load English tokenizer, tagger, parser and NER
    nlp = spacy.load("en_core_web_sm")
except IOError as e:
    print(f"Error loading spaCy model: {e}")
    # Handle the error gracefully, such as loading an alternative model or exiting the program
    exit(1)

# Read input text file
input_file_path = "output.txt_final"  # Change this to your input file path
try:
    with open(input_file_path, "r", encoding="utf-8") as file:
        text = file.read()
except UnicodeDecodeError:
    print("UnicodeDecodeError: Unable to decode the file using UTF-8 encoding.")
    # Handle the decoding error gracefully, such as trying a different encoding or exiting the program
    exit(1)

# Chunk size for processing
chunk_size = 100000  # Adjust as needed based on your text length and available memory

# Split the text into chunks
chunks = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

# Apply the model to each chunk
for chunk in chunks:
    doc = nlp(chunk)
    
    # Output file path for each chunk
    output_file_path = "output_chunk.txt"  # Change this to your output file path for each chunk
    
    # Write the entities identified by the model to the output file for each chunk
    with open(output_file_path, "a", encoding="utf-8") as output_file:
        for ent in doc.ents:
            output_file.write(f"{ent.text} - {ent.label_}\n")


In [ ]:
# objective 3
# classifying extracted legal keywords into crime categories 
 
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder \
    .appName("FIR Classification and Punishment Recommendation") \
    .getOrCreate()

# Read FIR data from input file
# Read FIR data from input file
input_file_path = "C:\\Users\\akank\\OneDrive\\Desktop\\ML March\\output_chunk.txt"  # Replace with your input file path
fir_data = spark.read.text(input_file_path)


# Function to classify criminal acts based on identified IPC sections
def classify_criminal_act(description):
    # Add your logic to classify criminal acts based on the description and identified IPC sections
    # Example:
    if "theft" in description.lower():
        return "Theft"
    elif "assault" in description.lower():
        return "Assault"
    else:
        return "Other"

# Function to recommend potential punishments based on classified acts and relevant IPC sections
def recommend_punishment(criminal_act):
    # Add your logic to recommend potential punishments based on the classified act and relevant IPC sections
    # Example:
    if criminal_act == "Theft":
      return "Imprisonment up to 3 years or fine or both (IPC Section 379)"
    elif criminal_act == "Assault":
        return "Imprisonment up to 1 year or fine or both (IPC Section 352)"
    else:
        return "Punishment not specified"


from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# Define the classification function as a UDF
classify_criminal_act_udf = udf(classify_criminal_act, StringType())

# Apply classification function to FIR data
classified_fir_data = fir_data.withColumn("Criminal_Act", classify_criminal_act_udf(fir_data["value"]))

# Output the results
classified_fir_data.show(truncate=False)


# Output the results
classified_fir_data.show(truncate=False)

# Stop Spark session
spark.stop()